In [2]:
!pip install dash
!pip install plotly
!pip install pandas
!pip install wget

from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import wget

data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

app = Dash(__name__)

app.layout = [
    html.H1(children='SpaceX Launch Statistics', style={'textAlign':'center', 'color':'#000000', 'font-size': 30}),
    dcc.Dropdown(id='dropdown-statistics',
    options=[
            {'label': 'Launch Success', 'value': 'Launch Success'},
            {'label': 'Launch Success Ratio', 'value': 'Launch Success Ratio'},
            {'label': 'Payload vs. Launch Outcome', 'value': 'PvLO'}],
    placeholder='Select a visualisation',
    ),
    dcc.RangeSlider(
        id='payload-slider',
        min=spacex_df["Payload Mass (kg)"].min(),
        max=spacex_df["Payload Mass (kg)"].max(),
        step=100,
        marks={int(i): f'{i}' for i in range(int(spacex_df["Payload Mass (kg)"].min()), int(spacex_df["Payload Mass (kg)"].max()), 1000)},
        value=[spacex_df["Payload Mass (kg)"].min(), spacex_df["Payload Mass (kg)"].max()],
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    dcc.Graph(id='graph-content')
]

@app.callback(
    Output(component_id='payload-slider', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))
    
def update_input_container(option):
    if option =='PvLO': 
        return False
    else: 
        return True

@app.callback(
    Output('graph-content', 'figure'),
    [Input(component_id='dropdown-statistics', component_property='value'), 
     Input('payload-slider', 'value')]
)
def update_output_container(option, selected_payload_range):
    # Print in debugging
    print(f'Stat chosen: {option} | Payload range: {selected_payload_range}')
    
    # Filter DataFrame based on the selected payload range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= selected_payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= selected_payload_range[1])]

    if option == 'PvLO':  # Payload vs Landing Outcome scatter plot
        # Create a scatter plot using Plotly
        fig = px.scatter(
            filtered_df, 
            x='Payload Mass (kg)', 
            y='Landing Outcome', 
            color='Launch Site',
            title=f'Payload vs. Landing Outcome for Payloads between {selected_payload_range[0]} and {selected_payload_range[1]}',
            labels={'PAYLOAD_MASS__KG_': 'Payload Mass (kg)', 'Landing Outcome': 'Landing Outcome'},
            height=600
        )
        return fig

    elif option == 'Launch Success Ratio':  # Pie chart for launch site with highest success ratio
        # Calculate success ratio for each launch site
        success_counts = spacex_df[spacex_df['Landing Outcome'].str.contains('Success', case=False)].groupby('Launch Site').size()
        total_counts = spacex_df.groupby('Launch Site').size()
        success_ratio = (success_counts / total_counts).sort_values(ascending=False)
        
        # Get the site with the highest success ratio
        top_launch_site = success_ratio.idxmax()
        
        # Filter the DataFrame for the top launch site
        top_site_df = spacex_df[spacex_df['Launch Site'] == top_launch_site]
        
        # Get the outcome breakdown for the top launch site
        outcome_counts = top_site_df['Landing Outcome'].value_counts()

        # Create a pie chart for the outcome breakdown
        fig = px.pie(
            outcome_counts,
            values=outcome_counts.values,
            names=outcome_counts.index,
            title=f'Launch Outcomes for {top_launch_site} (Highest Success Ratio)',
            color_discrete_sequence=px.colors.sequential.RdBu
        )
        return fig

    elif option == 'Launch Success':  # Pie chart for success rate across all sites
        # Filter for successful and unsuccessful launches across all sites
        outcome_counts = spacex_df[spacex_df['Landing Outcome'].str.contains('Success', case=False)]
        success_counts = outcome_counts.groupby('Launch Site').size()
        # Create a pie chart for the success/failure breakdown across all launch sites
        fig = px.pie(
            names=success_counts.index,
            values=success_counts.values,
            title='Launch Success Count for All Sites',
            color_discrete_sequence=px.colors.sequential.RdBu
        )
        return fig

    # Default figure in case no option matches
    return px.scatter(
        filtered_df, 
        x='Payload Mass (kg)', 
        y='Landing Outcome', 
        color='Launch Site',
        title=f'Default Scatter Plot for Payloads between {selected_payload_range[0]} and {selected_payload_range[1]}',
        labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'Landing Outcome': 'Landing Outcome'},
        height=600
    )

if __name__ == '__main__':
    app.run(debug=True,port='8008')

100% [................................................................................] 7710 / 7710

Stat chosen: None | Payload range: [0, 9600]
Stat chosen: None | Payload range: [0, 9600]
Stat chosen: Launch Success Ratio | Payload range: [0, 9600]
Stat chosen: PvLO | Payload range: [0, 9600]
Stat chosen: PvLO | Payload range: [0, 5000]
Stat chosen: PvLO | Payload range: [1200, 5000]
Stat chosen: Launch Success Ratio | Payload range: [1200, 5000]
Stat chosen: Launch Success | Payload range: [1200, 5000]
Stat chosen: PvLO | Payload range: [1200, 5000]
Stat chosen: PvLO | Payload range: [2500, 5000]
Stat chosen: Launch Success Ratio | Payload range: [2500, 5000]
